# Submitting calculations
#### Create a structure, kpoints, and input parameters and submit a Quantum ESPRESSO (PW) calculation
Time: 3 mins

<div class="alert alert-box alert-info">
This example expects that you have already imported the sample data provided with the demos (to have the SSSP pseudopotential library).
</div>

In [ ]:
from aiida.backends.utils import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()

from aiida.orm.utils import load_node, CalculationFactory
from aiida.work.run import run
from aiida.orm.code import Code
from aiida.orm.data.base import Bool, Str
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.array.kpoints import KpointsData
from aiida.orm.data.structure import StructureData
from aiida_quantumespresso.utils.pseudopotential import validate_and_prepare_pseudos_inputs
from aiida_quantumespresso.utils.resources import get_default_options
from ase.spacegroup import crystal

import nglview
#import warnings
#warnings.filterwarnings('ignore')

#### Create a diamond cubic crystal structure

In [ ]:
# Define a structure, employing the Atomic Structure Environment library
alat = 3.568
ase_definition = crystal('C', [(0,0,0)], spacegroup=227, cellpar=[alat, alat, alat, 90, 90, 90], primitive_cell=True)*2
structure = StructureData(ase=ase_definition)
structure.store()

#### Show the already stored structure

In [ ]:
view = nglview.show_ase(structure.get_ase())
view

#### Create the _k_-points mesh

In [ ]:
# Define the k-points mesh
kpoints = KpointsData()
kpoints_mesh = [1, 1, 1]
kpoints.set_kpoints_mesh(kpoints_mesh)
kpoints.store()

#### List existing pseudopotential families

In [ ]:
!verdi data upf listfamilies

#### Define the calculation input parameters

In [ ]:
# Define the pseudo potential family and input parameters for pw.x
pseudo_family = 'SSSP'
parameters = {
    'CONTROL': {
        "calculation": "scf",
        'tstress': True,
    },
    'SYSTEM': {
        'ecutwfc': 40.,
        'ecutrho': 320.,
    },
    'ELECTRONS': {
        'conv_thr': 1.e-10,
    }
}

#### Select the code to use

In [ ]:
#### Select the code to use

from notebook_helpers import get_code_pwonly_dropdown
from IPython.display import display
code_group = get_code_pwonly_dropdown()
display(code_group)

In [ ]:
# Dictionary, where values are the code labels of each type of code required
# Here we require only PW
code_names = code_group.children[1].value
if code_names:
    code_name = code_names['pw']
    print "I will use the code '{}'".format(code_names['pw'])
else:
    print "No code found: the rest will crash. Select a code, or configure one first!"
    code_name = None
code = Code.get_from_string(code_name)

#### Run a PwCalculation with the inputs we define

<div class="alert alert-box alert-info">
Remember at this stage to check if the daemon is started, otherwise the calculation will never run<br>
<br>
To check the daemon status, run in a terminal `verdi daemon status`<br>
To start the daemon, run in a terminal `verdi daemon start`
</div>

In [ ]:
PwCalculation = CalculationFactory('quantumespresso.pw')

inputs = {
    'code': code,
    'structure': structure,
    'kpoints': kpoints,
    'parameters': ParameterData(dict=parameters),
    'pseudo': validate_and_prepare_pseudos_inputs(structure, pseudo_family=Str('SSSP')),
    '_options': get_default_options()
}

print('Running a {} pw.x calculation... '.format('scf'))
results, pk = run(PwCalculation.process(), _return_pid=True, **inputs)
calc = load_node(pk)
print('PwCalculation<{}> terminated with state: {}'.format(calc.pk, calc.get_state()))
print('\n{link:25s} {node}'.format(link='Output link', node='Node pk and type'))
print('{s}'.format(s='-'*60))
for link, node in sorted(calc.get_outputs(also_labels=True)):
    print('{:25s} <{}> {}'.format(link, node.pk, node.__class__.__name__))

#### There are convenient methods to directly access the results

In [ ]:
print "Total energy = {} {}".format(calc.res.energy, calc.res.energy_units)